<a href="https://colab.research.google.com/github/tvaditya/intro_ds_and_ml/blob/main/%5BML1%5D_Introdu%C3%A7%C3%A3o_ao_Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução ao Scikit-Learn

O projeto `Scikit-learn` visa providenciar uma biblioteca de machine learning para Python.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/1200px-Scikit_learn_logo_small.svg.png" width=400px ></center>

Pensada para casar com os pacotes científicos que orbitam em volta das bibliotecas `Numpy` e `Scipy`, a visão do Scikit-learn é a de ser uma ferramenta eficiente e estável, ao mesmo tempo que seja acessível aos não-experts em machine learning.

De acordo com a sua [documentação oficial](https://scikit-learn.org/stable/getting_started.html), o `Scikit-learn` é uma biblioteca *open source* que suporta aprendizado supervisionado e não supervisionado, além de prover várias ferramentas para *model fitting*, pré-processamento de dados, seleção de modelos e avaliação, entre outras várias utilidades.

Para aprofundar na arquitetura da biblioteca, recomendo ler uma das melhores referências que encontrei, o paper [API design for machine learning software: experiences from the scikit-learn project](https://arxiv.org/abs/1309.0238).

## Representação dos dados no `Scikit-learn`

A opção da equipe de desenvolvimento da biblioteca foi escolher a representação de dados que mais se pareça com a representação matricial. Na prática, os *datasets* são codificados de duas maneira possíveis:

1. Arrays multidimensionais do `Numpy`, para dados densos; e
2. Matrizes esparças do `Scipy`, para dados esparços.

De acordo com o paper referenciado anteriormente, a decisão por usar esse tipo de representação, mesmo havendo outras mais sofisticadas, foi devido ao fato de isso permitir contar com toda a eficiência das operações vetorizadas do `Numpy` e `Scipy` - além de deixar o código enxuto e legível.

In [ ]:
# importando o conjunto de dados
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/heart-disease-uci.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Como você já sabe, cada linha representa uma entrada única, um lançamento no banco de dados e cada coluna representa uma *feature* (variável, atributo).

No caso desse DataFrame, para criarmos um modelo de regressão, seria necessário dividí-lo em uma matriz `X` contendo as variáveis independentes (*features*) e um vetor alvo `y` contendo as variáveis dependentes (variáveis alvos).

In [ ]:
X = df.drop('num', axis=1)
y = df.num

print("df.shape:\t{}".format(df.shape))
print("X.shape:\t{}".format(X.shape))
print("y.shape:\t{}".format(y.shape))

df.shape:	(303, 14)
X.shape:	(303, 13)
y.shape:	(303,)


## Interfaces do `Scikit-learn`

O design da biblioteca é organizado em volta de [três APIs fundamentais](https://towardsdatascience.com/scikit-learn-design-principles-d1371958059b), pilares básicos do `Scikit-learn`: ***Estimator, Predictor*** e ***Transformer***, e é uma prova do poder que dos *design patterns*.

Tenha em mente que as interfaces não se excluem ou tem um limite bem definido. Na verdade, elas são complementares. Vamos conhecer as três interfaces.

### Estimator API

Fazem parte do coração do `Scikit-learn`. É por meio dessa interface que você vai instanciar os objetos e rodar o método `fit` para treinar seu modelo.

Quero reforçar aqui que todo algoritmo de machine learning da biblioteca, é implementado pela API Estimator.

[Como resumiu bem o autor Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html), autor do livro *Python Data Science Handbook*, os passos mais comuns para usar essa API são:

1. Escolher a classe do modelo, por meio da importação da *estimator class* apropriada dentro do `Scikit-learn`.
2. Escolher os  hiperparâmetros do modelo, instanciando essa classe com os valores desejados.
3. Separar os dados entre matriz de *features* e vetor alvo.
4. Fit do modelo, rodando o método `fit()` na instância do modelo.
5. Aplicar o modelo aos dados novos, que pode ser:
    * Para aprendizado supervisionado, comumente é utilizado o método `predict()`.
    * Para aprendizado não supervisionado, comumente transformamos ou inferimos as propriedades dos dados usando `transform()` ou `predict()`.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=10000, penalty='l2')
model.fit(X, y)

### Predictor API

O *predictor interface* adiciona uma camada de previsão, onde recebe um array *X_test* e produz uma previsão para *X_test*.

No caso do aprendizado supervisionado, ele retorna um label com a classe prevista ou o valor calculado pelo modelo de machine learning.

Alguns modelos também trazem o método `predict_proba`, para informar qual a probabilidadade daquele *input* ser mesmo da classe prevista.

Por fim, ainda de acordo com o [paper citado no inicio deste notebook](https://arxiv.org/abs/1309.0238), essa interface traz ainda uma função *score*, para mensurar a perfomance do modelo.

In [ ]:
import numpy as np
X_test = np.array([36, 1.0, 2.0, 146.9, 210.6, 1.0, 2.0, 183.8, 0.0, 2.1, 2.1, 0.0, 5.0]).reshape(1, -1)
model.predict(X_test)

### Transformer API

Como a maior parte dos dados reais irá demandar um tratamento (ou passar por um filtro), a *transformer interface* define o método `transform`, que recebe um input `X` e devolve um output transformado `X`.

Feature extraction, redução de dimensionalidade, padronização, normalização, entre outros, são todos fornecidos por essa API. Veja o exemplo abaixo.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaler = scaler.transform(X)

In [ ]:
X

In [ ]:
X_scaler